In [1]:
# Importamos las clases que se requieren para manejar los agentes (Agent) y su entorno (Model).
# Cada modelo puede contener múltiples agentes.
from mesa import Agent, Model 

# Debido a que necesitamos que existe un solo agente por celda, elegimos ''SingleGrid''.
from mesa.space import MultiGrid

# Con ''RandomActivation'', hacemos que todos los agentes se activen ''al mismo tiempo''.
from mesa.time import RandomActivation

# Haremos uso de ''DataCollector'' para obtener información de cada paso de la simulación.
from mesa.datacollection import DataCollector

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

#Importamos random para obtener coordenadas al azar
import random

import ipywidgets as widgets
from IPython.display import display

#TYPES
#Type 1 Food
#Type 2 deposit 

In [2]:
class Agent(Agent):
    def __init__(self, id, model):
        super().__init__(id, model)
        self.type
        self.has_food
        self.deposit_coordinates = None # Inicialmente no se conocen las coordenadas del depósito
        self.id
        self.position = (0, 0)  # La posición inicial del agente
    
    # Actualiza las coordenadas del depósito y notifica a otros agentes.
    def Update_Deposit_Coordinates(self, deposit_coordinates):
        
        self.deposit_coordinates = deposit_coordinates

        # Notifica a otros agentes sobre las nuevas coordenadas del depósito
        for agent in self.model.schedule.agents:
            if agent != self:
                agent.deposit_coordinates = deposit_coordinates
        
    #Mueve al agente hacia el depósito.
    def Move_To_Deposit(self):
        if self.deposit_coordinates is not None:
            deposit_row, deposit_col = self.deposit_coordinates
            current_row, current_col = self.position

            # Calcula la distancia euclidiana entre la posición actual y las coordenadas del depósito
            distance = math.sqrt((deposit_row - current_row)**2 + (deposit_col - current_col)**2)

            # Establece la velocidad del agente (puedes ajustar este valor según tus necesidades)
            speed = 1

            # Calcula las diferencias en las coordenadas para determinar la dirección del movimiento
            delta_row = (deposit_row - current_row) / distance * speed
            delta_col = (deposit_col - current_col) / distance * speed

            # Mueve al agente hacia las coordenadas del depósito
            self.position = (current_row + delta_row, current_col + delta_col)
        
    def UpdateFood():

    def MoveToFood():



IndentationError: expected an indented block after function definition on line 19 (784995009.py, line 22)

In [3]:
#En get_grid obtenemos los contenidos de nuestra celda y seteamos valores numéricos
#para después setear los colores en el grid

def get_grid(model):
    grid = np.zeros((model.grid.width, model.grid.height), dtype=int)
    
    for (content, (x, y)) in model.grid.coord_iter():

        if content:
            grid[x][y] = 3   #Agent

        elif model.get_type(x, y) == 1: 
            grid[x][y] = 1  #Food

        elif model.get_type(x, y) == 2: 
            grid[x][y] = 2  #Deposit
        
        else:
            grid[x][y] = 0  # Empty cell

    return grid


In [8]:
class FoodModel(Model):
    def __init__ (self, width, height, num_agents, num_food):
        self.num_agents = num_agents
        self.grid = MultiGrid(width, height, torus = False)
        self.schedule = RandomActivation(self)
        self.datacollector = DataCollector(model_reporters={"Grid": get_grid})  
        self.type = np.zeros((width, height))
        self.deposit = 0

        self.food_quantity = num_food

        #añadimos una id única a los agentes
        id = 0

        for i in range(num_agents):
            x, y = random.randrange(self.grid.width), random.randrange(self.grid.height)

            while not (self.grid.is_cell_empty((x, y))):
                 x, y = random.randrange(self.grid.width), random.randrange(self.grid.height)    
            
            Agent = Agent(id, self)
            self.grid.place_agent(Agent, (x, y))
            self.schedule.add(Agent)
            id = id + 1

        #Add random deposit coordinates
        x, y = random.randrange(self.grid.width), random.randrange(self.grid.height)
        if (self.grid.is_cell_empty((x, y))):
                self.type[x][y] = 1

             
    def step(self):
        self.step_call_count = 1

        if self.get_current_food() == 0:
            self.running = False

        elif (self.step_call_count % 5 == 0):
            self.generate_food()
        else:
            self.call_count += 1
            self.datacollector.collect(self)
            self.schedule.step() 

        step_call_count +=1

    def get_type(self, x,y):
        return self.type[x][y]
    
    def update_deposit(self):
        self.deposit += 1
        
    def generate_food(self):
        self.call_count += 1
        if self.call_count >= 47:
            self.timer.append_stdout("Function called 47 times. Stopping.") 

        for i in range(self.food_quantity):      
            x, y = random.randrange(self.grid.width), random.randrange(self.grid.height)
            while not (self.grid.is_cell_empty(x, y) & self.get_type(x,y)== 0):
                 x, y = random.randrange(self.grid.width), random.randrange(self.grid.height) 

        self.type[x][y] = 1

    def get_current_food(self):
        xd =1


In [9]:
WIDTH = 100
HEIGHT = 100
NUM_AGENTS = 5
NUM_FOOD = 47
# Definimos el número máximo de steps a correr
MAX_STEPS = 1500

# Registramos el tiempo de inicio y ejecutamos la simulación
start_time = time.time()

model = FoodModel(WIDTH, HEIGHT, NUM_AGENTS, NUM_FOOD)

widgets.interact(model.step)

for i in range(MAX_STEPS):
    model.step()

# Imprimimos el tiempo que le tomó correr al modelo.

print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))

UnboundLocalError: cannot access local variable 'Agent' where it is not associated with a value

In [ ]:
#Recolectamos los datos del grid 
all_grid = model.datacollector.get_model_vars_dataframe()

In [ ]:

fig, axs = plt.subplots(figsize=(7, 7))
axs.set_xticks([])
axs.set_yticks([])

# Definimos el color para la distinguir agentes, y celdas sucias
#el numero que seteamos anteriormente corresponde al valor del array
#de colores que queremos que se muestre 
cmap = plt.cm.colors.ListedColormap(['white', 'red', 'blue', 'green'])

patch = plt.imshow(all_grid.iloc[0][0], cmap=cmap, vmin=0, vmax=3)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    patch.set_clim(vmin=0, vmax=2)  #establecemos los limites del color

anim = animation.FuncAnimation(fig, animate, frames=MAX_STEPS)